<a href="https://colab.research.google.com/github/i-moes/TM10007_PROJECT/blob/master/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TM10007 Assignment Group 7

---



In [0]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/i-moes/TM10007_PROJECT.git

# Install packages
!pip install sklearn numpy matplotlib

In [0]:
# # General packages
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import datasets as ds
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
 
# Metrics
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import mean_absolute_error
# from sklearn.metrics import r2_score

# Classifiers
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

## Section 1: Data loading and cleaning

In this section the data is loaded and preprocessed.

In [0]:
## Load Data

from brats.load_data import load_data
data = load_data()

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')


## Clean Data

# Replace all zero values with NaN
data.replace(0, np.nan, inplace=True)
# Replace all zero division errors with NaN
data.replace('#DIV/0!', np.nan, inplace=True)
# Replace all inf values with NaN
data.replace(np.inf, np.nan, inplace=True)

# Remove column when >5% of values is NaN
data.dropna(axis = 1, thresh=0.95*len(data.index), inplace=True)

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

# Extract labels from dataframe
labels = data['label']
# Drop column containing patient labels for imputation
data_imp = data.drop(columns=['label'])

# Impute for NaN values
imputor = KNNImputer(n_neighbors=5, weights='uniform')
# this will look for all columns where we have NaN value and replace the NaN value with specified test statistic
array_imp = imputor.fit_transform(data_imp)
data_imp[:] = array_imp

print(f'The number of samples: {len(data_imp.index)}')
print(f'The number of columns: {len(data_imp.columns)}')


## Section 2: Splitting Data in Train and Test Set

The data is splitted into a train and test set. This is done using train_test_split from sklearn.model_selection. The test data contains 45% of the data. 

In [6]:
# splitting data in train and test set 
x_train, x_test, y_train, y_test = train_test_split(data_imp, labels, test_size = 0.45)
print(x_train.head())
print(y_train.head())

              VOLUME_NET  VOLUME_ED  ...  TGM_Cog_Z_1   TGM_T_1
ID                                   ...                       
TCGA-HT-7884     32725.0    20904.0  ...     0.063533  128.4680
TCGA-DU-6542     18873.0    39845.0  ...     0.067363  229.4810
TCGA-12-0616      7774.0    21238.0  ...     0.061374  279.1780
TCGA-76-6656      9087.0    88963.0  ...     0.120478  240.2820
TCGA-CS-5396     93298.0     7407.0  ...     0.087105   41.9047

[5 rows x 456 columns]
ID
TCGA-HT-7884    LGG
TCGA-DU-6542    LGG
TCGA-12-0616    GBM
TCGA-76-6656    GBM
TCGA-CS-5396    LGG
Name: label, dtype: object


## Section 3: Feature Extraction

Features are extracted from the train set with  ... 

In [0]:
# Feature extraction

## Section 4: Training Classifiers

In this section ... are trained on the train set using sklearn. 

In [0]:
# Classifiers

## Section 5: Performance Evaluation

Testing classifiers on test set. The following evaluation measures are evaluated:


*   Accuracy
*   F1-Score
*  AUC



In [0]:
# Metrics 